In [1]:
#用到的套件


from __future__ import print_function, division
import os  
import pandas as pd
import requests
import numpy as np
import re
import math
import torchvision
import cv2
import glob
import random
from pathlib import Path
from torchvision import datasets,transforms
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
import time
import argparse
from time import sleep
from tqdm import tqdm, trange
from PIL import Image
from torch.autograd import Variable
import torch.nn.functional as FUN
from scipy import io
import efficientnet_pytorch
import torchvision.transforms as T
import PIL
import pickle
import torchvision.datasets as dsets
from scipy.misc import imsave
import torch.nn.functional as F

In [4]:
#定義function

def load_file(filename):
    with open(filename, 'rb') as fo:
        data = pickle.load(fo, encoding='latin1')
    return data

# 解壓縮，返回解壓後的字典
def unpickle(file):
    fo = open(file, 'rb')
    dict = pickle.load(fo, encoding='latin1')
    fo.close()
    return dict

#補邊,填充成正方形，防止resize變形
def expend_img(img):
    '''
    :param img: 图片数据
    :return:
    '''
    fill_pix=[0,0,0] #填充色素，可自己設定
    h,w=img.shape[:2]
    if h>=w: #左右填充
        padd_width=int(h-w)//2
        padd_top,padd_bottom,padd_left,padd_right=0,0,padd_width,padd_width #各個方向的填充像素
    elif h<w: #上下填充
        padd_high=int(w-h)//2
        padd_top,padd_bottom,padd_left,padd_right=padd_high,padd_high,0,0 #各個方向的填充像素
    new_img = cv2.copyMakeBorder(img,padd_top,padd_bottom,padd_left,padd_right,cv2.BORDER_CONSTANT, value=fill_pix)
    return new_img

#對影像做基本的旋轉 和亮度 對比度 飽和度的調整
def expend1_img(img):
    '''
    :param img: 圖片數據
    :return:
    '''
    a = random.random()
    
    t2 = transforms.RandomHorizontalFlip(p=0.5)  # 水平镜像，p是機率
    t3 = transforms.RandomVerticalFlip(p=0.2) #垂直鏡像
    # print(type(img))
    img = t2(img)
    img = t3(img)
    if a<0.4:
        t1 = transforms.RandomRotation(45)  # 随機旋轉，旋轉範圍為【-45,45】
        img = t1(img)

    t4 = transforms.ColorJitter(brightness=(0.8,1.5), contrast=(0.8,1.5), saturation=(0.8,1.5))#調整亮度  對比度  飽和度
    img = t4(img)
    return img

#對圖片做透視轉換
def expend2_img(img):
    t = transforms.RandomPerspective(distortion_scale=0.6,p=1,interpolation = 2,fill=0) #圖片透視化
    img2 = t(img)
    return img2

#切分訓練集和測試集，並進行補邊處理
def split_train_test(img_dir,save_dir,train_val_num):
    '''
    :param img_dir: 原始图片路径，注意是所有类别所在文件夹的上一级目录
    :param save_dir: 保存图片路径
    :param train_val_num: 切分比例
    :return:
    '''

    img_dir_list=glob.glob(img_dir+os.sep+"*")#獲取每個類别所在的路徑（一個類别對應一個文件夾）
    for class_dir in img_dir_list:
        class_name=class_dir.split(os.sep)[-1] #獲取當前類别
        img_list=glob.glob(class_dir+os.sep+"*") #獲取每個類别文件夾下的所有圖片
        all_num=len(img_list) #獲取總個數
        train_list=random.sample(img_list,int(all_num*train_val_num)) #訓練集圖片所在路徑
        save_train=save_dir+os.sep+'train'+os.sep+class_name
        save_val=save_dir+os.sep+"val"+os.sep+class_name
        os.makedirs(save_train,exist_ok=True)
        os.makedirs(save_val,exist_ok=True) #建立對應的文件夾
        #保存切分好的數據集
        for imgpath in img_list:
            imgname=Path(imgpath).name #獲取文件名
            if imgpath in train_list:
                img=cv2.imread(imgpath)
                new_img=expend_img(img)
                cv2.imwrite(save_train+os.sep+imgname,new_img)
            else: #將除了訓練集意外的數據均視為驗證集
                img = cv2.imread(imgpath)
                new_img = expend_img(img)
                cv2.imwrite(save_val + os.sep + imgname, new_img)
                
    print("split train and val finished !")

#資料增強
def data_enhancement(img_dir,save_dir,train_val_num):

    img_dir_list=glob.glob(img_dir+os.sep+"*")#獲取每个類别所在的路徑（一個類别對應一個文件夾）
    for class_dir in img_dir_list:
        class_name=class_dir.split(os.sep)[-1] #獲取當前類别
        img_list=glob.glob(class_dir+os.sep+"*") #獲取每個類别文件夾下的所有圖片
        all_num=len(img_list) #獲取總個數
        train_list=random.sample(img_list,int(all_num*train_val_num)) #訓練集圖片所在路徑
        save_train=save_dir+os.sep+"train"+os.sep+class_name
        save_val=save_dir+os.sep+"val"+os.sep+class_name
        os.makedirs(save_train,exist_ok=True)
        os.makedirs(save_val,exist_ok=True) #建立對應的文件夾
        # print(class_name+" trian num",len(train_list))
        # print(class_name+" val num",all_num-len(train_list))
        #保存切分好的數據集
        for imgpath in img_list:
            imgname=Path(imgpath).name #獲取文件名
            if imgpath in train_list:
                img= Image.open(imgpath)
                for time in range(3):
                    new_img=expend1_img(img)
                    Image.Image.save(new_img,save_train+os.sep+str(time)+imgname)

#資料增強( 透視轉換 )
def perspective_transform(img_dir,save_dir,train_val_num):
    
    img_dir_list=glob.glob(img_dir+os.sep+"*")#獲取每个類别所在的路徑（一個類别對應一個文件夾）
    for class_dir in img_dir_list:
        class_name=class_dir.split(os.sep)[-1] #獲取當前類别
        img_list=glob.glob(class_dir+os.sep+"*") #獲取每個類别文件夾下的所有圖片
        all_num=len(img_list) #獲取總個數
        train_list=random.sample(img_list,int(all_num*train_val_num)) #訓練集圖片所在路徑
        save_train=save_dir+os.sep+"train"+os.sep+class_name
        save_val=save_dir+os.sep+"val"+os.sep+class_name
        os.makedirs(save_train,exist_ok=True)
        os.makedirs(save_val,exist_ok=True) #建立對應的文件夾
        # print(class_name+" trian num",len(train_list))
        # print(class_name+" val num",all_num-len(train_list))
        #保存切分好的數據集
        for imgpath in img_list:
            imgname=Path(imgpath).name #獲取文件名
            if imgpath in train_list:
                img= Image.open(imgpath)
                for time in range(1):
                    img2 = expend2_img(img)
                    Image.Image.save(img2,save_train+os.sep+str(time)+'_per_'+imgname)

#efficientionnet訓練模型
class Efficientnet_train():
    def __init__(self,opt):
        self.epochs=opt.epochs #訓練週期
        self.batch_size=opt.batch_size #batch_size
        self.class_num=opt.class_num #類别數
        self.imgsz=opt.imgsz #圖片尺寸
        self.img_dir=opt.img_dir #圖片路徑
        self.weights=opt.weights #模型路徑
        self.save_dir=opt.save_dir #保存模型路徑
        self.save_model_name=opt.save_model_name #保存模型檔名
        self.lr=opt.lr #初始化學習率
        self.moment=opt.m #動量
        base_model = EfficientNet.from_name('efficientnet-b5') #加載模型，使用b幾的就改為b幾
        state_dict = torch.load(self.weights)
        base_model.load_state_dict(state_dict)
        # 修改全連接層
        num_ftrs = base_model._fc.in_features
        base_model._fc = nn.Linear(num_ftrs, self.class_num)
        print(device)
        self.model = base_model.to(device)
        # 交叉熵損失函數
        self.cross = nn.CrossEntropyLoss()
        # 優化器
        self.optimzer = optim.SGD((self.model.parameters()), lr=self.lr, momentum=self.moment, weight_decay=0.0004)

        #獲取處理後的數據集和類别映射表
        self.trainx,self.valx,self.b=self.process()
        print(self.b)
    def __call__(self):
        best_acc = 0
        self.model.train(True)
        for ech in tqdm(range(self.epochs)):
            optimzer1 = self.lrfn(ech, self.optimzer)

            print("----------Start Train Epoch %d----------" % (ech + 1))
            # 開始訓練
            run_loss = 0.0  # 損失
            run_correct = 0.0  # 準確率
            count = 0.0  # 分類正確的個數

            for i, data in enumerate(self.trainx):
                # print('train')
                inputs, label = data
                inputs, label = inputs.to(device), label.to(device)

                # 訓練
                optimzer1.zero_grad()
                output = self.model(inputs)

                loss = self.cross(output, label)
                loss.backward()
                optimzer1.step()

                run_loss += loss.item()  # 損失累加
                _, pred = torch.max(output.data, 1)
                count += label.size(0)  # 求總共的訓練個數
                run_correct += pred.eq(label.data).cpu().sum()  # 截止當前預測正確的個數
                #每隔100個batch顯示一次信息，這裡顯示的ACC是當前預測正確的個數/當前訓練過的個數
                if (i+1)%100==0:
                    print('[Epoch:{}__iter:{}/{}] | Acc:{}'.format(ech + 1,i+1,len(self.trainx), run_correct/count))
            # print(run_correct,'------------',count)
            train_acc = run_correct / count
            # 每次訓完一批顯示一次信息
            print('Epoch:{} | Loss:{} | Acc:{}'.format(ech + 1, run_loss / len(self.trainx), train_acc))

            # 訓完一批次後進行驗證
            print("----------Waiting Test Epoch {}----------".format(ech + 1))
            with torch.no_grad():
                correct = 0.  # 預測正確的個數
                total = 0.  # 總個數
                for inputs, labels in self.valx:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = self.model(inputs)

                    # 穫取最高分的那個類的索引
                    _, pred = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += pred.eq(labels).cpu().sum()
                test_acc = correct / total
                print("批次%d的验证集准确率" % (ech + 1), correct / total)
            if best_acc < test_acc:
                best_acc = test_acc
                start_time=(time.strftime("%m%d",time.localtime()))
                save_weight=self.save_dir+os.sep+start_time #保存路徑
                os.makedirs(save_weight,exist_ok=True)
                torch.save(self.model.state_dict(), save_weight + os.sep + self.save_model_name)#不加state_dict()存法會直接把模型架構和權重一起存入weight檔中
                                                                                                       #加state_dict()則只單純存權重(不易報錯)

  #數據處理
    def process(self):
        # 數據增强
        data_transforms = {
            'train': transforms.Compose([
                transforms.Resize((self.imgsz, self.imgsz)),  # resize
                transforms.CenterCrop((self.imgsz, self.imgsz)),  # 中心裁剪
                transforms.RandomRotation(45),  # 随機旋轉，旋轉範圍為【-45,45】
                transforms.ToTensor(),  # 轉換為張量
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 標準化
            ]),
            "val": transforms.Compose([
                transforms.Resize((self.imgsz, self.imgsz)),  # resize
                transforms.CenterCrop((self.imgsz, self.imgsz)),  # 中心裁剪
                transforms.ToTensor(),  # 張量轉換
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        }

        # 定義圖像生成器
        image_datasets = {x: datasets.ImageFolder(root=os.path.join(self.img_dir,x), transform=data_transforms[x]) for x in ['train', 'val']}

        # 得到訓練集和驗證集
        trainx = DataLoader(image_datasets["train"], batch_size=self.batch_size, shuffle=True, drop_last=True)
        valx = DataLoader(image_datasets["val"], batch_size=self.batch_size, shuffle=True, drop_last=False)
        b = image_datasets["train"].class_to_idx  # id和類别對
        return trainx,valx,b


    # 學習率慢热加下降
    def lrfn(self,num_epoch, optimzer):
        lr_start = 0.00001  # 初始值
        max_lr = 0.0004  # 最大值
        lr_up_epoch = 10  # 學習率上升10个epoch
        lr_sustain_epoch = 5  # 學習率保持不變
        lr_exp = .8  # 衰减因子
        if num_epoch < lr_up_epoch:  # 0-10个epoch學習率線性增加
            lr = (max_lr - lr_start) / lr_up_epoch * num_epoch + lr_start
        elif num_epoch < lr_up_epoch + lr_sustain_epoch:  # 學習率保持不變
            lr = max_lr
        else:  # 指數下降
            lr = (max_lr - lr_start) * lr_exp ** (num_epoch - lr_up_epoch - lr_sustain_epoch) + lr_start
        for param_group in optimzer.param_groups:
            param_group['lr'] = lr
        return optimzer

#分類網路 function

class Residual(nn.Module):  
    def __init__(self, in_channels, out_channels):
        super(Residual, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=1)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=2, stride=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.bn2 = nn.BatchNorm1d(out_channels)
        
        self.conv4res = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=2, stride=1)
        self.AvgPool1d=nn.AdaptiveAvgPool1d(12)
        self.bn4res = nn.BatchNorm1d(out_channels)

    def forward(self, X):
        
        Y1 = self.conv4res(X)
        Y1 = self.bn4res(Y1)

        Y2 = self.bn1(self.conv1(X))

        Y2 = F.relu(Y2)

        Y2 = self.bn2(self.conv2(Y2))

        Y2 = F.relu(Y2)

        return Y1 + Y2

def resnet_block(in_channels, out_channels, num_residuals):

    blk = []
    for i in range(num_residuals):
        blk.append(Residual(in_channels, out_channels))

    return nn.Sequential(*blk)

class LSTM_FCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(LSTM_FCN, self).__init__()
        # LSTM
        self.conv4lstm = nn.Conv1d(48, 48, kernel_size=2, stride=2)
        self.rnn = torch.nn.LSTM(input_dim, hidden_dim, num_layers=layers, batch_first=True)

        # 1D conv
        self.conv1 = nn.Conv1d(1, 256, kernel_size=2, stride=2)
        self.res_block=resnet_block(in_channels=256,out_channels=256,num_residuals=3)
        self.conv2 = nn.Conv1d(256, 16, kernel_size=1, stride=1)
        self.AvgPool1d=nn.AdaptiveAvgPool1d(48) # length

        # concat softmax
        self.fc1 = nn.Linear(768, 256, bias=True)
        self.dropou1=nn.Dropout(0.7)
        self.fc2 = nn.Linear(256, 64, bias=False)
        self.dropou2=nn.Dropout(0.7)
        self.fc3 = nn.Linear(64, output_dim, bias=False)
        self.softmax=nn.Softmax(dim=1)

    def init_model(self):
        for m in self.modules():
            if isinstance(m, (torch.nn.Linear, torch.nn.Conv1d)):
                torch.nn.init.xavier_uniform_(m.weight)
        print("init success !!")

    def forward(self, x):
        
        
        
        # x=torch.unsqueeze(x,0)
        # 1D cnn
        cnn_out=self.conv1(x)
        cnn_out=self.res_block(cnn_out)
        cnn_out=self.conv2(cnn_out)
        cnn_out=self.AvgPool1d(cnn_out)            

        y=torch.flatten(cnn_out,start_dim=1)
   
        y = self.fc1(y) 
        y=self.dropou1(y)
  
        y = self.fc2(y) 
        y=self.dropou2(y)
        y = self.fc3(y) 

        y= self.softmax(y)
 
        return y

'''算entropy'''
def entropy(input):
    all = 0
    for t in range(len(input)):
        if input[t]>=0:
            en = -(input[t]*math.log(input[t],2))
            all = all+en
    return all


# Load Test images
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor


class ImageFolderWithPaths(datasets.ImageFolder):
    def __init__(self, *args):
        super(ImageFolderWithPaths, self).__init__(*args)
        self.trans = args[1]
    def __len__(self):
      return len(self.imgs)
    def __getitem__(self, index):
        img, label = super(ImageFolderWithPaths, self).__getitem__(index)
        
        path = self.imgs[index][0]
        return (img, label ,path)

In [ ]:
#將data分到對應的類別的資料夾

img_dir = 'cifar_100/all'
img_path_list = glob.glob(img_dir+os.sep+"*")
img_list= os.listdir(img_dir)
save_dir = 'cifar_100/train_class'

# for t in range(100):
#     os.makedirs(save_dir+'/'+str(t),exist_ok=True)

with open('cifar_100/train_label.txt','r') as f :
    img_path = f.read()

img_path_list = img_path.splitlines()

count = 0
for path in img_path_list:
    a = path.split(' ')
    img=cv2.imread(img_dir+'/'+a[0]+'.png')
    try:
        new_img=expend_img(img)
    except:
        pass
    cv2.imwrite(save_dir+'/'+a[1]+'/'+a[0]+'.png',new_img)
    count+=1

In [9]:
#將train資料集取出10%，和5% 5%分開

img_dir = 'cifar_100/train_class/train'
save_dir = 'cifar_100\cifar100_50percent_og/train'
save_dir2 = 'cifar_100\cifar100_50percent_og/train'
train_val_num = 0.5

img_dir_list=glob.glob(img_dir+os.sep+"*")#獲取每個類别所在的路徑（一個類别對應一個文件夾）
for class_dir in img_dir_list:
    class_name=class_dir.split(os.sep)[-1] #獲取當前類别
    img_list=glob.glob(class_dir+os.sep+"*") #獲取每個類别文件夾下的所有圖片
    all_num=len(img_list) #獲取總個數
    train_list=random.sample(img_list,int(all_num*train_val_num)) #訓練集圖片所在路徑
    save_one=save_dir+os.sep+os.sep+class_name
    save_two=save_dir2+os.sep+os.sep+class_name
    os.makedirs(save_one,exist_ok=True)
    os.makedirs(save_two,exist_ok=True)


    #保存切分好的數據集
    for imgpath in img_list:
        imgname=Path(imgpath).name #獲取文件名
        if imgpath in train_list:
            img=cv2.imread(imgpath)
            new_img=expend_img(img)
            cv2.imwrite(save_one+os.sep+imgname,new_img)
        # else:
        #     img=cv2.imread(imgpath)
        #     new_img=expend_img(img)
        #     cv2.imwrite(save_two+os.sep+imgname,new_img)

In [ ]:
#將資料集切成train和val

img_dir = 'cifar_10/register'
save_dir = 'cifar_10/register_split' 
# img_dir = 'imagenette_10percent/second_5percent'
# save_dir = 'imagenette_10percent/second_5percent_split' 
train_val_num = 0.7
split_train_test(img_dir,save_dir,train_val_num)

In [ ]:
#圖片資料增強

img_dir = 'cifar_10\cifar10_50percent_split/train'
save_dir = 'cifar_10\cifar10_50percent_split' 
train_val_num = 1.0
data_enhancement(img_dir,save_dir,train_val_num)
perspective_transform(img_dir,save_dir,train_val_num)

In [ ]:
#訓練efficientionnet

device="cuda" if torch.cuda.is_available() else "cpu"

#参數設置
def parse_opt():
    parser=argparse.ArgumentParser()
    parser.add_argument("--weights",type=str,default="./model/efficientnet-b5-b6417697.pth",help='initial weights path')#預訓練模型路徑
    parser.add_argument("--img-dir",type=str,default="./cifar_100/cifar100_50percent_just_good_model",help="train image path") #數據集的路徑
    parser.add_argument("--imgsz",type=int,default=224,help="image size") #圖像尺寸
    parser.add_argument("--epochs",type=int,default=50,help="train epochs")#訓練批次
    parser.add_argument("--batch-size",type=int,default=8,help="train batch-size") #batch-size
    parser.add_argument("--class_num",type=int,default=100,help="class num") #類別數
    parser.add_argument("--lr",type=float,default=0.0005,help="Init lr") #學習率初始值
    parser.add_argument("--m",type=float,default=0.9,help="optimer momentum") #動量
    parser.add_argument("--save-dir",type=str,default="./weight",help="save models dir")#保存模型路徑

    parser.add_argument("--save-model-name",type=str,default="efficientb5_cifar100_50percent_just_good_model.pth",help="save models name")#保存模型路徑
    
    opt=parser.parse_known_args()[0]
    return opt

if __name__ == '__main__':
    opt=parse_opt()
    models=Efficientnet_train(opt)
    models()

In [ ]:
#生成dict，內容 : ( 各類的cofidence , gt label  ) 

tensor = []
label = []
class_num = []
path = []

input_size = 224
device = 'cuda'
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor
means = [0.485, 0.456, 0.406]
stds = [0.229, 0.224, 0.225]
unorm = UnNormalize(mean = means, std = stds)
class ImageFolderWithPaths(datasets.ImageFolder):
    def __init__(self, *args):
        super(ImageFolderWithPaths, self).__init__(*args)
        self.trans = args[1]
    def __len__(self):
      return len(self.imgs)
    def __getitem__(self, index):
        img, label = super(ImageFolderWithPaths, self).__getitem__(index)
        
        path = self.imgs[index][0]
        return (img, label ,path)
# Load Test images
def loaddata(data_dir, batch_size, set_name, shuffle):
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(means, stds)
        ]),
        'val': transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize(means, stds)
        ]),
        'test_class': transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize(means, stds)
        ]),
    }

    image_datasets = {x: ImageFolderWithPaths(os.path.join(data_dir, x), data_transforms[x]) for x in [set_name]}
    # num_workers=0 if CPU else = 1
    dataset_loaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                      batch_size=batch_size,
                                                      shuffle=shuffle, num_workers=0) for x in [set_name]}
    data_set_sizes = len(image_datasets[set_name])
    return dataset_loaders, data_set_sizes


def test_model(model, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    cont = 0
    outPre = []
    initi_tensor = []
    outLabel = []
    img_path = []
    dset_loaders, dset_sizes = loaddata(data_dir=data_dir, batch_size=batch_size, set_name='train', shuffle=False)
    transform = T.ToPILImage()
    for data in dset_loaders['train']:
        inputs, labels, paths = data #path抓出被分類的圖片的原始路徑
        labels = labels.type(torch.LongTensor)
        

        # GPU
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())

        outputs = model(inputs)

        tensor.append(outputs.data)

        _, preds = torch.max(outputs.data, 1)

        class_num.append(labels)

        path.append(paths)

        loss = criterion(outputs, labels)
        if cont == 0:
            outPre = outputs.data.cpu()
            outLabel = labels.data.cpu()
        else:
            outPre = torch.cat((outPre, outputs.data.cpu()), 0)
            outLabel = torch.cat((outLabel, labels.data.cpu()), 0)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        cont += len(labels)
        acc = running_corrects/cont

    return FUN.softmax(Variable(outPre)).data.numpy(), outLabel.numpy()


if __name__ == '__main__':
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    # Start Testing
    net_name = 'efficientnet-b5'
    data_dir = 'cifar_100/train_class'
    save_dir = 'weight/0803'
    modelft_file = save_dir + "/" + 'efficientb5_cifar100_100percent' + '.pth'
    batch_size = 1

    # GPU時
    model_ft = efficientnet_pytorch.EfficientNet.from_name(net_name)
    # 修改全連接層
    num_ftrs = model_ft._fc.in_features
    model_ft._fc = nn.Linear(num_ftrs, 100)
    model_ft = model_ft.to(device)

    model_ft.load_state_dict(torch.load(modelft_file))
    print(type(model_ft))
    criterion = nn.CrossEntropyLoss().cuda()
    outPre, outLabel = test_model(model_ft, criterion)


print(str(len(tensor)),'----',str(len(class_num)),str(len(path)))

with open('pickle/cifar100_100percent/cifar100_all_tensor.pickle', 'wb') as f:
    pickle.dump(tensor, f)

with open('pickle/cifar100_100percent/cifar100_all_class_label.pickle', 'wb') as f:
    pickle.dump(class_num, f)

with open('pickle/cifar100_100percent/cifar100_all_path.pickle', 'wb') as f:
    pickle.dump(path, f)

In [11]:
#生成資料夾

path = 'pickle/cifar100_test/class'
for t in range(100):    
    os.makedirs(path+str(t))

In [ ]:
#製作label和資料集 ( 只取最大的值 )

#分類標準 0:上升 1:不變 2:下降

with open('pickle/cifar100_train_tensor.pickle', 'rb') as f:
    train_tensor = pickle.load(f)
with open('pickle/cifar100_train_label.pickle', 'rb') as f:
    train_label = pickle.load(f)
with open('pickle/cifar100_val_tensor.pickle', 'rb') as f:
    val_tensor = pickle.load(f)
with open('pickle/cifar100_val_label.pickle', 'rb') as f:
    val_label = pickle.load(f)

path = 'pickle/cifar100_test'
class_num = 100
for class_ in range(class_num):

    t_label = []
    t_data = []
    v_label = []
    v_data = []
    count0 = 0
    count1 = 0
    count2 = 0
    count3 = 0
    count4 = 0
    # os.makedirs(path+'/'+'class'+str(class_))

    for t in range(len(train_tensor)):
        value , index = torch.sort(train_tensor[t].squeeze(),descending = True)
        # for t2 in range(int(len(index)/33)):
        for t2 in range(int(len(index)/20)):

            if int(index[t2]) == class_:
                

                # tensor = torch.empty(1,9)
                # for t3 in range(9):
                #     tensor[0][t3] = (train_tensor[t][0][index[t2]] - train_tensor[t][0][index[t2+t3+1]]) / train_tensor[t][0][index[t2]]
                # t_data.append(tensor)

                t_data.append(train_tensor[t])

                differ = train_label[t][0][class_] - train_tensor[t][0][class_]
                if differ >= 0:
                    increase = abs(differ) / abs(train_tensor[t][0][class_])

                    if increase > 1 :
                        classifi = 0
                        count0+=1

                    elif increase<=1:
                        classifi = 1
                        count1+=1

                else:
                    reduce = abs(differ) / abs(train_tensor[t][0][class_])

                    if reduce > 0.5 :
                        classifi = 2
                        count2+=1

                    elif reduce<=0.5:
                        classifi = 1
                        count1+=1

                t_label.append(torch.tensor(classifi))
    
    da = torch.stack(t_data)
    la = torch.stack(t_label)
    print(len(t_data),'---',len(t_label))
    print(count0,'--',count1,'--',count2,'--',count3,'--',count4)
    with open(path+'/'+'class'+str(class_)+'/cifar100_class'+str(class_)+'_label.pickle', 'wb') as f:
        pickle.dump(la, f)
    with open(path+'/'+'class'+str(class_)+'/cifar100_class'+str(class_)+'_tensor.pickle', 'wb') as f:
        pickle.dump(da, f)


    for t in range(len(val_tensor)):
        value , index = torch.sort(val_tensor[t].squeeze(),descending = True)
        # for t2 in range(int(len(index)/33)):
        for t2 in range(int(len(index)/20)):

            if int(index[t2]) == class_:


                # tensor = torch.empty(1,9)
                # for t3 in range(9):
                #     tensor[0][t3] = (val_tensor[t][0][index[t2]] - val_tensor[t][0][index[t2+t3+1]]) / val_tensor[t][0][index[t2]]
                # t_data.append(tensor)
                # v_data.append(tensor)

                v_data.append(val_tensor[t])

                differ = val_label[t][0][class_] - val_tensor[t][0][class_]
                if differ >= 0:
                    increase = abs(differ) / abs(val_tensor[t][0][class_])

                    if increase > 1 :
                        classifi = 0
                        count0+=1

                    elif increase<=1:
                        classifi = 1
                        count1+=1

                else:
                    reduce = abs(differ) / abs(val_tensor[t][0][class_])

                    if reduce > 0.5 :
                        classifi = 2
                        count2+=1

                    elif reduce<=0.5:
                        classifi = 1
                        count1+=1

                v_label.append(torch.tensor(classifi))
    
    da2 = torch.stack(v_data)
    la2 = torch.stack(v_label)
    print(len(v_data),'---',len(v_label))
    print(count0,'--',count1,'--',count2,'--',count3,'--',count4)
    with open(path+'/'+'class'+str(class_)+'/cifar100_val_class'+str(class_)+'_label.pickle', 'wb') as f:
        pickle.dump(la2, f)
    with open(path+'/'+'class'+str(class_)+'/cifar100_val_class'+str(class_)+'_tensor.pickle', 'wb') as f:
        pickle.dump(da2, f)

In [51]:
#定義分類模型

device="cuda" if torch.cuda.is_available() else "cpu"
MyResModel = LSTM_FCN(input_dim=1, hidden_dim=32, output_dim=3, layers=1).to(device)
MyResModel.init_model()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(MyResModel.parameters(), lr=0.001, momentum=0.8)

init success !!


In [52]:
# 學習率慢热加下降
def lrfn(num_epoch, optimzer):
    lr_start = 0.001  # 初始值
    max_lr = 0.005  # 最大值
    lr_up_epoch =150  # 學習率上升10个epoch
    lr_sustain_epoch = 50  # 學習率保持不變
    lr_exp = .8  # 衰减因子
    if num_epoch < lr_up_epoch:  # 0-10个epoch學習率線性增加
        lr = (max_lr - lr_start) / lr_up_epoch * num_epoch + lr_start
    elif num_epoch < lr_up_epoch + lr_sustain_epoch:  # 學習率保持不變
        lr = max_lr
    else:  # 指數下降
        lr = (max_lr - lr_start) * lr_exp ** (num_epoch - lr_up_epoch - lr_sustain_epoch) + lr_start
    for param_group in optimzer.param_groups:
        param_group['lr'] = lr
    return optimzer

In [53]:
#train分類模型

Epochs=800
max_test_acc=0.0
class_number = 100
train_loss_list=[]
train_acc_list=[]
test_loss_list=[]
test_acc_list=[]
batch_size = 32
device = 'cuda'
model_path = 'linear_weight/cifar100_ece'
test = []

'''匯入所有label'''
train_label_ = []
for t in range(class_number):
    with open('pickle/cifar100_test/class'+str(t)+'/cifar100_class'+str(t)+'_label.pickle', 'rb') as f:
        train_label_.append(pickle.load(f))


val_label_ = []
for t in range(class_number):
    with open('pickle/cifar100_test/class'+str(t)+'/cifar100_val_class'+str(t)+'_label.pickle', 'rb') as f:
        val_label_.append(pickle.load(f))


'''匯入所有confidence'''
train_tensor_ = []
for t in range(class_number):
    with open('pickle/cifar100_test/class'+str(t)+'/cifar100_class'+str(t)+'_tensor.pickle', 'rb') as f:
        train_tensor_.append(pickle.load(f))


val_tensor_ = []
for t in range(class_number):
    with open('pickle/cifar100_test/class'+str(t)+'/cifar100_val_class'+str(t)+'_tensor.pickle', 'rb') as f:
        val_tensor_.append(pickle.load(f))


#訓練各類模型
for class_n in range(class_number):
    print('class '+str(class_n)+' model')

    # 按batch size包裝training set資料
    register_1 = []
    register_2 = []
    train_tensor_list = []
    train_label_list = []
    count = 0
    best_acc = 0
    for t in range(len(train_tensor_[class_n])):
        count+=1

        #不做confidence calibration
        # numpy = FUN.softmax(Variable(train_tensor_[class_n][t])).data.cpu().numpy()#把input從confidence轉為機率

        #做confidence calibration
        caliration = train_tensor_[class_n][t]/0.5
        soft = FUN.softmax(Variable(caliration)).data.cpu()#把input從confidence轉為機率
        numpy = soft.numpy()

        # 對input做confidence calibration( 只取confidence高的 )
        value , index = torch.sort(train_tensor_[class_n][t].squeeze(),descending = True)
        for t2 in range(len(index)):
            if t2>20:
                pass
            else:
                numpy[0][index[t2]] = numpy[0][index[t2]]


        tensor = torch.tensor(numpy)
        # tensor = train_tensor_[class_n][t]
        register_1.append(tensor)
        register_2.append(train_label_[class_n][t])
        if count % batch_size ==0:
            a = torch.stack(register_1)
            b = torch.stack(register_2)
            train_tensor_list.append(a)
            train_label_list.append(b)
            register_1 = []
            register_2 = []

    #依bach size打包validation set
    register_1 = []
    register_2 = []
    val_tensor_list = []
    val_label_list = []
    count = 0
    for t in range(len(val_tensor_[class_n])):
        count+=1

        #不做confidence calibration
        #numpy = FUN.softmax(Variable(val_tensor_[class_n][t])).data.cpu().numpy()#把input從confidence轉為機率

        #做confidence calibration
        caliration = val_tensor_[class_n][t]/0.5
        soft = FUN.softmax(Variable(caliration)).data.cpu()#把input從confidence轉為機率
        numpy = soft.numpy()


        # 對input做confidence calibration( 只取confidence高的 )
        value , index = torch.sort(val_tensor_[class_n][t].squeeze(),descending = True)
        for t2 in range(len(index)):
            if t2>20:
                pass
            else:
                numpy[0][index[t2]] = numpy[0][index[t2]]



        tensor = torch.tensor(numpy)
        # tensor = val_tensor_[class_n][t]
        register_1.append(tensor)
        register_2.append(val_label_[class_n][t])
        if count % batch_size ==0:
            a = torch.stack(register_1)
            b = torch.stack(register_2)
            val_tensor_list.append(a)
            val_label_list.append(b)
            register_1 = []
            register_2 = []


    register = 0
    #開始訓練模型
    for epoch in range(Epochs):
        optimizer1 = lrfn(epoch,optimizer)
        train_loss=0.0
        train_acc=0.0
        test_loss=0.0
        test_acc=0.0
        MyResModel.train()
        count = 0
        train_loss_count = 0
        train_acc_count = 0
        for data_ in train_tensor_list:
            optimizer1.zero_grad()
            pred = MyResModel(data_.to(device))

            max_,class_ = torch.max(pred.data,1)
            train_correct = (class_==train_label_list[count].to(device)).sum()
            train_acc = train_correct / batch_size
            train_acc_count = train_acc_count + train_acc.item()

            loss = criterion(pred.to(device),train_label_list[count].to(device))
            train_loss_count = train_loss_count+loss.item()

            count+=1
            loss.backward()
            optimizer1.step()
        MyResModel.eval()
        a = (epoch+1) / 100

        if count ==0 :
            count = 1

        if (epoch+1) % 10 ==0:
            pass
            # print('epoch : '+ str(int(a*100)) +' train_loss : '+str(train_loss_count/count)+' train_Acc : '+str(train_acc_count/count))

        #算驗證集loss和acc
        if (epoch+1) % 100 ==0:
            count = 0
            loss_count = 0
            acc_count = 0
            for data_ in val_tensor_list:
                pred = MyResModel(data_.to(device))
                max_,class_ = torch.max(pred.data,1)
                correct = (class_==val_label_list[count].to(device)).sum()
                acc = correct / batch_size
                acc_count = acc_count + acc.item()
                loss = criterion(pred.to(device),val_label_list[count].to(device))
                loss_count = loss_count+loss.item()
                count+=1

            if count ==0 :
                count = 1

            if acc_count/count >= best_acc:
                best_acc = acc_count/count
                torch.save(MyResModel.state_dict(), model_path+'/'+'model'+str(class_n)+'.pth')
                a = (epoch+1) / 100
                print('epoch : '+ str(a*100) +' loss : '+str(loss_count/count)+' Acc : '+str(acc_count/count))
                register = acc_count/count

                
    if register >=0.7:
        test.append(str(class_n)) #test是紀錄線性模型中準確率好的class
with open('pickle/good_model/good_model_list_ece.pickle', 'wb') as f:
    pickle.dump(test, f)   

class 0 model


c:\Users\Taka\anaconda3\envs\taka1\lib\site-packages\ipykernel_launcher.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
c:\Users\Taka\anaconda3\envs\taka1\lib\site-packages\ipykernel_launcher.py:98: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epoch : 100.0 loss : 0.7075719331440172 Acc : 0.84375
class 1 model
epoch : 100.0 loss : 0.9098750352859497 Acc : 0.625
epoch : 300.0 loss : 0.8637825151284536 Acc : 0.6875
epoch : 600.0 loss : 0.8681372702121735 Acc : 0.6875
epoch : 700.0 loss : 0.8637609283129374 Acc : 0.6875
class 2 model
epoch : 100.0 loss : 0.9228382110595703 Acc : 0.6276041666666666
epoch : 200.0 loss : 0.8903159896532694 Acc : 0.6614583333333334
epoch : 400.0 loss : 0.8794773519039154 Acc : 0.6692708333333334
epoch : 600.0 loss : 0.8395985017220179 Acc : 0.7109375
class 3 model
epoch : 100.0 loss : 1.0627556890249252 Acc : 0.48828125
epoch : 200.0 loss : 1.030042752623558 Acc : 0.51953125
epoch : 300.0 loss : 0.7577744722366333 Acc : 0.79296875
epoch : 400.0 loss : 0.7453185543417931 Acc : 0.8046875
epoch : 800.0 loss : 0.7437660321593285 Acc : 0.80859375
class 4 model
epoch : 100.0 loss : 1.0194397270679474 Acc : 0.53125
epoch : 200.0 loss : 1.0117085576057434 Acc : 0.5390625
class 5 model
epoch : 100.0 loss : 

In [39]:
#生成一個效能好的model的list
with open('pickle/good_model/good_model_list_ece.pickle', 'rb') as f:  
    good_model = pickle.load(f)

print(len(good_model))
    

44


In [40]:
#用train出的模型修改本來的confidence( 修改前5大的 )

with open('pickle/cifar100_all/cifar100_all_tensor.pickle', 'rb') as f:  
    result = pickle.load(f)

with open('pickle\cifar100_all/cifar100_all_class_label.pickle', 'rb') as f:
    label = pickle.load(f)

with open('pickle\cifar100_all/cifar100_all_path.pickle', 'rb') as f:
    path = pickle.load(f)


weight_dir = 'linear_weight/cifar100_test'
weight_path = os.listdir(weight_dir)
model_file = []
count0 = 0
count1 = 0
count2 = 0
count3 = 0
count4 = 0

for t in range(len(weight_path)):
    path = weight_dir + '/' + weight_path[t]
    model_file.append(path)

for t in tqdm(range(len(result))):     
    value , index = torch.sort(result[t].squeeze(),descending = True)
    for t2 in range(int(len(index)/20)):
        
        MyResModel.load_state_dict(torch.load(model_file[int(index[t2])]))

        MyResModel = MyResModel.to(device)
        numpy = FUN.softmax(Variable(result[t])).data.cpu().numpy()
        tensor = torch.tensor(numpy)
        tensor = tensor.unsqueeze(0)
        tensor = tensor.to(device)
        output = MyResModel(tensor)
        pre_class = torch.argmax(output)


        #只修改模型效果好的

        if index[t2] in good_model:
            if int(pre_class)==0:
                result[t][0][int(index[t2])] = result[t][0][int(index[t2])] + result[t][0][int(index[t2])]*0.5
                count0+=1
            if int(pre_class)==1:
                result[t][0][int(index[t2])] = result[t][0][int(index[t2])]
                count1+=1
            if int(pre_class)==2:
                result[t][0][int(index[t2])] = result[t][0][int(index[t2])]- result[t][0][int(index[t2])]*0.2
                count2+=1




acc_count = 0
for t in range(len(result)):        
        r = torch.argmax(result[t])
        if int(r) == label[t]:
            acc_count+=1
print(acc_count)
print('acc : '+str(acc_count/len(result)))

with open('pickle/cifar100_all/alldata_revise_max5_good_model_warmup.pickle', 'wb') as f:
    pickle.dump(result, f)       

100%|██████████| 50000/50000 [1:46:34<00:00,  7.82it/s]


31052
acc : 0.62104


In [7]:
#用改完的confidence算enstropy挑 x% data(各類數量平均)

with open('pickle\cifar100_all/cifar100_all_class_label.pickle', 'rb') as f:
    label = pickle.load(f)

with open('pickle\cifar100_all/cifar100_all_path.pickle', 'rb') as f:
    path = pickle.load(f)

with open('pickle\cifar100_all/alldata_revise_max5_just_good_model.pickle', 'rb') as f:
    result = pickle.load(f)

class_num = 100



img_save_dir = 'cifar_100/cifar100_50percent_just_good_model/train' 
img_dir = 'cifar_100/train_class/train'
all_en = []
for t in range(len(result)):
    odds = FUN.softmax(Variable(result[t]).cpu()).data.numpy()
    odds = odds.reshape([class_num])
    data_entropy = entropy(odds)
    all_en.append(data_entropy)
sort = sorted(range(len(all_en)) , reverse = True,key = lambda k : all_en[k])

img_dir_list=glob.glob(img_dir+os.sep+"*")#獲取每個類别所在的路徑（一個類别對應一個文件夾）
for class_dir in img_dir_list:
    class_name=class_dir.split(os.sep)[-1] #獲取當前類别
    save_train=img_save_dir+os.sep+os.sep+class_name
    os.makedirs(save_train,exist_ok=True)#建立對應的文件夾

con = [0 for t in range(class_num)]
for t in range(int(len(sort))):
    p = re.sub("\,","",str(path [sort[t]]))
    p = re.sub("\(","",p)
    p = re.sub("\)","",p)
    p = re.sub("\'","",p)
    split = p.split('\\')

    if con[int(split[4])] <= (int(len(sort))/class_num) *0.5:

        con[int(split[4])] = con[int(split[4])]+1
        img = cv2.imread(p)
        cv2.imwrite(img_save_dir+'/'+ split[4]+'/'+split[6], img)

c:\Users\Taka\anaconda3\envs\taka1\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [54]:
s = 'ssss'
a = s
print(a)

ssss
